### Output Parser (for agents)
For structured output and peace of mind!


https://python.langchain.com/docs/how_to/structured_output/#advanced-raw-outputs


Gemini official doc suggests not to use schema in prompt which the output parser from Langchain does.
check this out -> https://ai.google.dev/gemini-api/docs/structured-output
**How does Google manages output parsing?**

Gemini already has capability to generate structured output. We can use this, but issue I am facing is how to use it with cra.

Check this tutorial doc as well -> https://langchain-ai.github.io/langgraph/agents/agents/#5-add-memory (but this has a cons - it calls the LLM again at the end with message to generate this output)

https://github.com/langchain-ai/langgraph/discussions/3574


### Final Thoughts 
1. creating structured outputs with different LLMs is a complex task and need specific solutions
2. LangGraph `create_react_agent` does provide option to pass structure but this calls LLM separately for structured output response in the end.
3. `Gemini` has `with_structured_output` method, we can use this, but (so far) I am not able to get a way to make it work in single LLM call, so I will also use separate LLM call for this. This looks safer also but has a cost factor too. So for now, lets use this and later experiment more on this.

In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from IPython.display import Markdown
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent

In [2]:
from typing import List

In [9]:
# parser

class MyOutput(BaseModel):
    place: str = Field(description="Place name")
    temperature: str = Field(description="temperature")
    # questions: List[Question] = Field(
    #     description="A list of quiz questions, mixing MCQ and short answer types. Ensure 10 questions total."
    # )


output_parser = PydanticOutputParser(pydantic_object=MyOutput)

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

search = DuckDuckGoSearchRun()

search.invoke("What's the weather currently in Raipur?")

"Raipur, India - Current temperature and weather conditions. Detailed hourly weather forecast for today - including weather conditions, temperature, pressure, humidity, precipitation, dewpoint, wind, visibility, and UV index data. How's the weather today in Raipur, Chhattisgarh. We provide information on: wind speed, wind direction, atmospheric pressure, temperature, humidity and visibility. Raipur Weather Forecast. Providing a local hourly Raipur weather forecast of rain, sun, wind, humidity and temperature. The Long-range 12 day forecast also includes detail for Raipur weather today. Live weather reports from Raipur weather stations and weather warnings that include risk of thunder, high UV index and forecast gales. Weather in Chhattisgarh, Raipur: Get the current weather, hourly and weekly weather forecast for Raipur, along with current temperature, rainfall, wind speed, humidity, air-quality, 15-days weather forecast and season trend for Raipur. Detailed weather forecast for Raipur

In [5]:
load_dotenv()

True

In [6]:
model = init_chat_model(
    "gemini-2.0-flash", model_provider="google_genai")

# .with_structured_output(MyOutput)
response = model.invoke("Hello World!!")
print(response)

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--379e5e8f-36b9-475f-93f9-ba15ca31cf26-0' usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}}


In [7]:
print(type(response))
# response.model_dump()
response.content

<class 'langchain_core.messages.ai.AIMessage'>


'Hello there! How can I help you today?'

In [8]:
tools = [search]
agent_executor = create_react_agent(model, tools)

In [10]:
agent = create_react_agent(
    model=model,
    tools=[search],
    response_format=MyOutput
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

In [13]:
response["structured_response"].model_dump()

{'place': 'San Francisco', 'temperature': 'unknown'}

In [146]:
raw_template = """
You are a helpful AI assistant. Use the following tools to answer questions.
you should use the 'DuckDuckGo' tool to get most relevant answers.
Answer concisely and directly based on the search results. Include links of sources in answers for user to access them directly.
     
     question : {topic}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Answer the user query. you should use the 'DuckDuckGo' tool to get most relevant answers.
            Wrap the output in `json` tags\n{format_instructions}""",
            
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=output_parser.get_format_instructions())
# This is adding the format instructions in the prompt itself.

query = "What is the rating for movie Materialist"

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. you should use the 'DuckDuckGo' tool to get most relevant answers.
            Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"description": "A suitable title.", "title": "Title", "type": "string"}, "description": {"description": "A brief description.", "title": "Description", "type": "string"}, "rating": {"description": "rating.", "title": "Rating", "type": "integer"}}, "required": ["title", "description", "rating"]}
```
Human: What is the rating for movie Materialist


In [147]:
# response = agent_executor.invoke({"messages": [
#                                  messages]})


In [148]:
chain = prompt | agent_executor 

In [149]:
response = chain.invoke({"query": query})

In [150]:
response["messages"]

[SystemMessage(content='Answer the user query. you should use the \'DuckDuckGo\' tool to get most relevant answers.\n            Wrap the output in `json` tags\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "A suitable title.", "title": "Title", "type": "string"}, "description": {"description": "A brief description.", "title": "Description", "type": "string"}, "rating": {"description": "rating.", "title": "Rating", "type": "integer"}}, "required": ["title", "description", "rating"]}\n```', additional_kwargs={}, response_metadata={}, id='fc87

In [151]:
response

{'messages': [SystemMessage(content='Answer the user query. you should use the \'DuckDuckGo\' tool to get most relevant answers.\n            Wrap the output in `json` tags\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "A suitable title.", "title": "Title", "type": "string"}, "description": {"description": "A brief description.", "title": "Description", "type": "string"}, "rating": {"description": "rating.", "title": "Rating", "type": "integer"}}, "required": ["title", "description", "rating"]}\n```', additional_kwargs={}, response_metadata

In [152]:
response["messages"][2]

AIMessage(content='```json\n{\n  "title": "I am unable to find the rating of the movie Materialist.",\n  "description": "I am unable to find the rating of the movie Materialist using the available tools.",\n  "rating": 0\n}\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e84813a5-f6a5-418b-b3c8-7a5f1d673fe6-0', usage_metadata={'input_tokens': 286, 'output_tokens': 57, 'total_tokens': 343, 'input_token_details': {'cache_read': 0}})

Output parsing a ChatMessage is different than parsing a direct message from AI. The chat message contains a list of system, user then AI message so explicitly pass the AI message to parser.

In [153]:
output = output_parser.parse(response["messages"][2].content)

In [154]:
output.model_dump()

{'title': 'I am unable to find the rating of the movie Materialist.',
 'description': 'I am unable to find the rating of the movie Materialist using the available tools.',
 'rating': 0}

### For the course recommendation output

In [23]:
# parser

class Recommendation(BaseModel):
    title: str = Field(description="A title for this recommendation.")
    description: str = Field(description="A brief description.")
    url: str = Field(
        description="source URL of this recommendation if available else NULL.")

class RecommendationList(BaseModel):
    recommendations: List[Recommendation] = Field(description="List of recommendations")
    

output_parser = PydanticOutputParser(pydantic_object=RecommendationList)

In [156]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful AI assistant. Create a recommendation list for the student to master the provided topic by user.
                Supplement your answer with the search results from 'DuckDuckGo' tool to get most relevant recommendations for the topic.
                Include links of sources in answers for user to access them directly.
                Answer maximum 10 recommendations.
                """,

        ),
        ("user", "{topic}"),
    ]
)

# .partial(format_instructions=output_parser.get_format_instructions())

topic = "Linear Algebra"

print(prompt.invoke({"topic": topic}))

messages=[SystemMessage(content="You are a helpful AI assistant. Create a recommendation list for the student to master the provided topic by user.\n                Supplement your answer with the search results from 'DuckDuckGo' tool to get most relevant recommendations for the topic.\n                Include links of sources in answers for user to access them directly.\n                Answer maximum 10 recommendations.\n                ", additional_kwargs={}, response_metadata={}), HumanMessage(content='Linear Algebra', additional_kwargs={}, response_metadata={})]


In [157]:
recommend_chain = prompt | agent_executor

In [158]:
response = recommend_chain.invoke({"topic": topic})

In [159]:
output_parser.parse(response["messages"][2].content)

OutputParserException: Invalid json output: Okay, I can create a recommendation list to help a student master Linear Algebra. I'll use DuckDuckGo to find some helpful resources.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [ ]:
response["messages"]

[SystemMessage(content="You are a helpful AI assistant. Create a recommendation list for the student to master the provided topic by user.\n                Supplement your answer with the search results from 'DuckDuckGo' tool to get most relevant recommendations for the topic.\n                Include links of sources in answers for user to access them directly.\n                Answer maximum 10 recommendations.\n                ", additional_kwargs={}, response_metadata={}, id='ecf7cf06-f408-4373-925d-4ecf8fbc18c6'),
 HumanMessage(content='Linear Algebra', additional_kwargs={}, response_metadata={}, id='aff4893c-c167-451b-810b-7012bc6a298b'),
 AIMessage(content='Okay, here\'s a recommendation list to help you master Linear Algebra, incorporating resources found via DuckDuckGo search:\n\n1.  **Textbook:** "Linear Algebra and Its Applications" by Gilbert Strang. This is a classic textbook that provides a comprehensive introduction to the subject. Many online courses also follow this bo

In [ ]:
Markdown(response["messages"][2].content)

Okay, here's a recommendation list to help you master Linear Algebra, incorporating resources found via DuckDuckGo search:

1.  **Textbook:** "Linear Algebra and Its Applications" by Gilbert Strang. This is a classic textbook that provides a comprehensive introduction to the subject. Many online courses also follow this book.

2.  **MIT OpenCourseWare:** Gilbert Strang's Linear Algebra course on MIT OpenCourseWare ([https://ocw.mit.edu/courses/18-06-linear-algebra-spring-2010/](https://ocw.mit.edu/courses/18-06-linear-algebra-spring-2010/)). It includes video lectures, lecture notes, problem sets, and solutions.

3.  **Khan Academy:** Khan Academy offers a free Linear Algebra course covering a wide range of topics, from basic matrix operations to more advanced concepts like eigenvalues and eigenvectors ([https://www.khanacademy.org/math/linear-algebra](https://www.khanacademy.org/math/linear-algebra)).

4.  **3Blue1Brown's "Essence of Linear Algebra" series:** This YouTube series provides a visual and intuitive understanding of the fundamental concepts of linear algebra ([https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab](https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)).

5.  **Practice Problems:** "Schaum's Outline of Linear Algebra" by Seymour Lipschutz and Marc Lipson. This book provides a large number of solved problems and practice exercises.

6.  **Interactive Linear Algebra:** This is a free online textbook with interactive examples and exercises ([http://textbooks.math.gatech.edu/ila/](http://textbooks.math.gatech.edu/ila/)).

7.  **Coding Implementation:** Implement linear algebra concepts in Python using NumPy. This will solidify your understanding and help you see how these concepts are used in practice.

8.  **Linear Algebra Done Right by Sheldon Axler:** This book takes a more abstract and theoretical approach, which can be beneficial for a deeper understanding.

9. **Applications:** Explore the applications of linear algebra in fields like computer graphics, machine learning, and data science. Seeing how the concepts are used in real-world applications can increase your motivation and understanding.

10. **Math Stack Exchange:** Use Math Stack Exchange to ask questions and get help from other students and experts.

In [ ]:
structured_output_model = init_chat_model(
    "gemini-2.0-flash", model_provider="google_genai").with_structured_output(RecommendationList)

In [ ]:
# Prompt for the structured output conversion
structured_output_prompt = ChatPromptTemplate.from_messages([
    ("system", """
     You are a data formatting assistant. Create a response in JSON format for the user input.
     Use the 'RecommendationList' schema for response wit each item having title, description and url.
     Ensure all fields are correctly populated based on the input text.
     """),
    ("user", "{agent_output_text}")
])

In [ ]:
structured_chain = structured_output_prompt | structured_output_model

In [ ]:
final_output = structured_chain.invoke(
    {"agent_output_text": response["messages"][2].content})

In [160]:
final_output

RecommendationList(recommendations=[Recommendation(title='Linear Algebra and Its Applications by Gilbert Strang', description='This is a classic textbook that provides a comprehensive introduction to the subject. Many online courses also follow this book.', sourceURL=0), Recommendation(title="MIT OpenCourseWare: Gilbert Strang's Linear Algebra course", description='It includes video lectures, lecture notes, problem sets, and solutions.', sourceURL=1), Recommendation(title='Khan Academy Linear Algebra course', description='Khan Academy offers a free Linear Algebra course covering a wide range of topics, from basic matrix operations to more advanced concepts like eigenvalues and eigenvectors', sourceURL=2), Recommendation(title="3Blue1Brown's Essence of Linear Algebra series", description='This YouTube series provides a visual and intuitive understanding of the fundamental concepts of linear algebra', sourceURL=3), Recommendation(title="Schaum's Outline of Linear Algebra by Seymour Lipsc

In [161]:
print(type(final_output.model_dump()))
final_output.model_dump()

<class 'dict'>


{'recommendations': [{'title': 'Linear Algebra and Its Applications by Gilbert Strang',
   'description': 'This is a classic textbook that provides a comprehensive introduction to the subject. Many online courses also follow this book.',
   'sourceURL': 0},
  {'title': "MIT OpenCourseWare: Gilbert Strang's Linear Algebra course",
   'description': 'It includes video lectures, lecture notes, problem sets, and solutions.',
   'sourceURL': 1},
  {'title': 'Khan Academy Linear Algebra course',
   'description': 'Khan Academy offers a free Linear Algebra course covering a wide range of topics, from basic matrix operations to more advanced concepts like eigenvalues and eigenvectors',
   'sourceURL': 2},
  {'title': "3Blue1Brown's Essence of Linear Algebra series",
   'description': 'This YouTube series provides a visual and intuitive understanding of the fundamental concepts of linear algebra',
   'sourceURL': 3},
  {'title': "Schaum's Outline of Linear Algebra by Seymour Lipschutz and Marc 

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful AI assistant. Create a recommendation list for the student to master the provided topic by user.
                Supplement your answer with the search results from 'DuckDuckGo' tool to get most relevant recommendations for the topic.
                Include links of sources in answers for user to access them directly.
                Answer maximum 10 recommendations.
                """,

        ),
        ("user", "{topic}"),
    ]
)

# .partial(format_instructions=output_parser.get_format_instructions())

topic = "Linear Algebra"

print(prompt.invoke({"topic": topic}))

messages=[SystemMessage(content="You are a helpful AI assistant. Create a recommendation list for the student to master the provided topic by user.\n                Supplement your answer with the search results from 'DuckDuckGo' tool to get most relevant recommendations for the topic.\n                Include links of sources in answers for user to access them directly.\n                Answer maximum 10 recommendations.\n                ", additional_kwargs={}, response_metadata={}), HumanMessage(content='Linear Algebra', additional_kwargs={}, response_metadata={})]


In [24]:
recommender_agent = create_react_agent(
    model=model,
    tools=[search],
    response_format=RecommendationList
)

In [25]:
rec_chain = prompt | recommender_agent

In [26]:
response = rec_chain.invoke({"topic": topic})

In [27]:
response["messages"]

[SystemMessage(content="You are a helpful AI assistant. Create a recommendation list for the student to master the provided topic by user.\n                Supplement your answer with the search results from 'DuckDuckGo' tool to get most relevant recommendations for the topic.\n                Include links of sources in answers for user to access them directly.\n                Answer maximum 10 recommendations.\n                ", additional_kwargs={}, response_metadata={}, id='c45839b8-450a-4d74-94be-a7a3278ba4f9'),
 HumanMessage(content='Linear Algebra', additional_kwargs={}, response_metadata={}, id='42173b39-4e31-449e-a5a6-5569cd55f9c0'),
 AIMessage(content='Okay, here\'s a recommendation list to help you master Linear Algebra, incorporating resources found via DuckDuckGo:\n\n1.  **Start with the Basics:** Ensure you have a solid foundation in basic algebra and arithmetic. This includes understanding variables, equations, and basic mathematical operations.\n\n2.  **Khan Academy\'

In [28]:
response["structured_response"].model_dump()

{'recommendations': [{'title': 'Review Basic Algebra',
   'description': 'Start with the basics of algebra.',
   'url': ''},
  {'title': 'Khan Academy Linear Algebra',
   'description': 'Khan Academy is a free resource that covers the fundamentals of linear algebra with videos and exercises.',
   'url': 'https://www.khanacademy.org/math/linear-algebra'},
  {'title': "MIT OpenCourseWare: Gilbert Strang's Linear Algebra",
   'description': 'A classic and comprehensive resource with lectures and course materials available online.',
   'url': "Search 'MIT OpenCourseWare Linear Algebra Gilbert Strang'"},
  {'title': "'Linear Algebra and Its Applications' by Gilbert Strang",
   'description': 'A widely used textbook that provides a clear and thorough introduction to linear algebra.',
   'url': ''},
  {'title': "'Linear Algebra Done Right' by Sheldon Axler",
   'description': 'This book takes a more abstract approach, focusing on vector spaces and linear operators.',
   'url': ''},
  {'title'